# importing the libraries

In [487]:
import numpy as np
import pandas as pd

# loading the dataset 

In [488]:
movie=pd.read_csv('MoviesOnStreamingPlatforms_updated.csv')


# understanding the dataset

In [489]:
movie.columns

Index(['Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors',
       'Genres', 'Country', 'Language', 'Runtime'],
      dtype='object')

In [490]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16744 non-null  int64  
 1   ID               16744 non-null  int64  
 2   Title            16744 non-null  object 
 3   Year             16744 non-null  int64  
 4   Age              7354 non-null   object 
 5   IMDb             16173 non-null  float64
 6   Rotten Tomatoes  5158 non-null   object 
 7   Netflix          16744 non-null  int64  
 8   Hulu             16744 non-null  int64  
 9   Prime Video      16744 non-null  int64  
 10  Disney+          16744 non-null  int64  
 11  Type             16744 non-null  int64  
 12  Directors        16018 non-null  object 
 13  Genres           16469 non-null  object 
 14  Country          16309 non-null  object 
 15  Language         16130 non-null  object 
 16  Runtime          16152 non-null  float64
dtypes: float64(2

observation: 


# Data cleaning

In [491]:
movie=movie.drop(columns=['Unnamed: 0']) # removing the unnecessary columns

In [492]:
movie['Age']=movie['Age'].fillna(movie['Age'].mode()[0])
movie['IMDb']=movie['IMDb'].fillna(movie['IMDb'].mean()).round(1)

movie['Rotten Tomatoes']=movie['Rotten Tomatoes'].str.strip('%')
movie['Rotten Tomatoes']=movie['Rotten Tomatoes'].astype(float)
movie['Rotten Tomatoes']=(movie['Rotten Tomatoes'].fillna(movie['Rotten Tomatoes'].mean())/10).round(1)

In [493]:
movie['Directors']=movie['Directors'].fillna('unknown')
movie['Genres']=movie['Genres'].fillna('unknown')
movie['Country']=movie['Country'].fillna('unknown')
movie['Language']=movie['Language'].fillna('unknown')
movie['Runtime']=movie['Runtime'].fillna(movie['Runtime'].mean())

In [494]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               16744 non-null  int64  
 1   Title            16744 non-null  object 
 2   Year             16744 non-null  int64  
 3   Age              16744 non-null  object 
 4   IMDb             16744 non-null  float64
 5   Rotten Tomatoes  16744 non-null  float64
 6   Netflix          16744 non-null  int64  
 7   Hulu             16744 non-null  int64  
 8   Prime Video      16744 non-null  int64  
 9   Disney+          16744 non-null  int64  
 10  Type             16744 non-null  int64  
 11  Directors        16744 non-null  object 
 12  Genres           16744 non-null  object 
 13  Country          16744 non-null  object 
 14  Language         16744 non-null  object 
 15  Runtime          16744 non-null  float64
dtypes: float64(3), int64(7), object(6)
memory usage: 2.0+ MB


In [495]:
movie.head(2)

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,8.7,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,8.7,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0


In [496]:
x=movie['Genres'].str.split(',')
Genres=x.explode()

Genres_df=pd.merge(movie[['Title','IMDb','Runtime']],Genres,left_index=True,right_index=True)
Genres_df.head()

,Title,IMDb,Runtime,Genres
0,Inception,8.8,148.0,Action
0,Inception,8.8,148.0,Adventure
0,Inception,8.8,148.0,Sci-Fi
0,Inception,8.8,148.0,Thriller
1,The Matrix,8.7,136.0,Action


In [497]:
# Splitting 'Language' column where multiple languages are separated by commas
language_split = movie['Language'].str.split(',')
Languages = language_split.explode()

# Creating a DataFrame with Title, IMDb, and Language
Languages_df = pd.merge(movie[['Title', 'IMDb']], Languages, left_index=True, right_index=True)
Languages_df.head()


,Title,IMDb,Language
0,Inception,8.8,English
0,Inception,8.8,Japanese
0,Inception,8.8,French
1,The Matrix,8.7,English
2,Avengers: Infinity War,8.5,English


In [498]:
# Melting the dataset to transform platform columns into rows
Platforms_df = movie.melt(id_vars=['Title', 'IMDb'], 
                          value_vars=['Netflix', 'Hulu', 'Prime Video', 'Disney+'], 
                          var_name='Platform', 
                          value_name='Available')

# Filtering out movies that are NOT available (Available == 0)
Platforms_df = Platforms_df[Platforms_df['Available'] == 1].drop(columns=['Available'])

Platforms_df.head()
Platforms_df
#------------------------
Platforms_df1 = movie.melt(id_vars=['Title', 'IMDb','Year','Rotten Tomatoes'], 
                          value_vars=['Netflix', 'Hulu', 'Prime Video', 'Disney+'], 
                          var_name='Platform', 
                          value_name='Available')
Platforms_df1 = Platforms_df1[Platforms_df1['Available'] == 1].drop(columns=['Available'])

Platforms_df1.head()
Platforms_df1


,Title,IMDb,Year,Rotten Tomatoes,Platform
0,Inception,8.8,2010,8.7,Netflix
1,The Matrix,8.7,1999,8.7,Netflix
2,Avengers: Infinity War,8.5,2018,8.4,Netflix
3,Back to the Future,8.5,1985,9.6,Netflix
4,"The Good, the Bad and the Ugly",8.8,1966,9.7,Netflix
...,...,...,...,...,...
66971,The Ghosts of Buxley Hall,6.2,1980,6.5,Disney+
66972,The Poof Point,4.7,2001,6.5,Disney+
66973,Sharks of Lost Island,5.7,2013,6.5,Disney+
66974,Man Among Cheetahs,6.6,2017,6.5,Disney+


In [499]:
# Splitting 'Directors' column
directors_split = movie['Directors'].str.split(',')
Directors = directors_split.explode()

# Creating Directors_df
Directors_df = pd.merge(movie[['Title', 'IMDb']], Directors, left_index=True, right_index=True)
Directors_df.head()

,Title,IMDb,Directors
0,Inception,8.8,Christopher Nolan
1,The Matrix,8.7,Lana Wachowski
1,The Matrix,8.7,Lilly Wachowski
2,Avengers: Infinity War,8.5,Anthony Russo
2,Avengers: Infinity War,8.5,Joe Russo


In [500]:
#Splitting 'Country' column
Country_spilt=movie["Country"].str.split(',')
Country=Country_spilt.explode("Country_spilt")

#creating Country_df
Country_df=pd.merge(movie[['Title', 'IMDb','Genres']], Country, left_index=True, right_index=True)
Country_df.head()

,Title,IMDb,Genres,Country
0,Inception,8.8,"Action,Adventure,Sci-Fi,Thriller",United States
1,The Matrix,8.7,"Action,Sci-Fi",United Kingdom
2,Avengers: Infinity War,8.5,"Action,Adventure,Sci-Fi",United States
3,Back to the Future,8.5,"Adventure,Comedy,Sci-Fi",United States
4,"The Good, the Bad and the Ugly",8.8,Western,United States


In [501]:
# Defining runtime categories
bins = [0, 60, 120, 180, 300]  # (in minutes)
labels = ['Short (<60 min)', 'Medium (60-120 min)', 'Long (120-180 min)', 'Very Long (>180 min)']
movie['Runtime Category'] = pd.cut(movie['Runtime'], bins=bins, labels=labels, right=False)


In [502]:
movie

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Runtime Category
0,1,Inception,2010,13+,8.8,8.7,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.000000,Long (120-180 min)
1,2,The Matrix,1999,18+,8.7,8.7,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.000000,Long (120-180 min)
2,3,Avengers: Infinity War,2018,13+,8.5,8.4,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.000000,Long (120-180 min)
3,4,Back to the Future,1985,7+,8.5,9.6,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.000000,Medium (60-120 min)
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,9.7,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.000000,Long (120-180 min)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,16740,The Ghosts of Buxley Hall,1980,18+,6.2,6.5,0,0,0,1,0,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English,120.000000,Long (120-180 min)
16740,16741,The Poof Point,2001,7+,4.7,6.5,0,0,0,1,0,Neal Israel,"Comedy,Family,Sci-Fi",United States,English,90.000000,Medium (60-120 min)
16741,16742,Sharks of Lost Island,2013,18+,5.7,6.5,0,0,0,1,0,Neil Gelinas,Documentary,United States,English,93.413447,Medium (60-120 min)
16742,16743,Man Among Cheetahs,2017,18+,6.6,6.5,0,0,0,1,0,Richard Slater-Jones,Documentary,United States,English,93.413447,Medium (60-120 min)


# DATA ANALYSIS

# How many unique movies are in the dataset?

In [503]:

# pd.unique(movie['Title'])
movie['Title'].nunique()

16744

# what is the average IMDB rating of all movies

In [504]:

movie.loc[::,['IMDb']].mean()
#------------------------OR
movie['IMDb'].mean()

np.float64(5.902657668418538)

In [505]:
len(movie)

16744

# how many movies were released each year

In [506]:



gp=movie.groupby(movie['Year'])
gp['Year'].count()
# .sort_values(ascending=False)




Year
1902       1
1912       1
1913       2
1915       4
1916       1
        ... 
2016    1206
2017    1401
2018    1285
2019     689
2020     147
Name: Year, Length: 109, dtype: int64

# what is the most common genre in the data set

In [507]:

Genres_df['Genres'].mode()[0]
#---------------or----------
movie['Genres'].mode()[0]


'Drama'

# what is the most common language for movies

In [508]:

Languages_df['Language'].mode()[0]
#--------------or
movie['Language'].mode()[0]

'English'

In [509]:
movie.columns

Index(['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors', 'Genres',
       'Country', 'Language', 'Runtime', 'Runtime Category'],
      dtype='object')

# How many movies are available on each streaming platform?

In [510]:

gp=Platforms_df.groupby('Platform')
gp['Platform'].count()
#-------or----------
movie[['Netflix','Hulu', 'Prime Video', 'Disney+']].sum()
#----------------------------or---------
Platforms_df['Platform'].value_counts()
#----------------or------------------
plts=movie[['Netflix','Hulu', 'Prime Video', 'Disney+']]
plts[plts==1].count()



Netflix         3560
Hulu             903
Prime Video    12354
Disney+          564
dtype: int64

# Which platform has the highest average IMDb rating?

In [511]:

# gp=movie.groupby(['Netflix','Hulu', 'Prime Video', 'Disney+'])
# gp['IMDb'].mean().idxmax()
#------------------------------------
gp=Platforms_df.groupby('Platform')
gp['IMDb'].mean().idxmax()

'Disney+'

In [512]:
movie.columns

Index(['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors', 'Genres',
       'Country', 'Language', 'Runtime', 'Runtime Category'],
      dtype='object')

# Are there any movies available on all four platforms (Netflix, Hulu, Prime Video, Disney+)?

In [513]:



movie[movie[['Netflix','Hulu', 'Prime Video', 'Disney+']].sum(axis=1)==4]

# movie.loc[Platforms_df['Title'].isin('gp'),['Title']]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Runtime Category


observation:

# Which platform has the most movies with an IMDb rating above 8.0?

In [514]:

# cond=movie['IMDb']>8.0
# movie.loc[cond,['Title']]
#-----------or--------------
cond=Platforms_df['IMDb']>8.0
Platforms_df.loc[cond,['Platform']].value_counts().head(1)
#--------------
Platforms_df.loc[cond,['Platform']].mode()['Platform'][0]

'Prime Video'

# Which platform has the most movies released after 2015?

In [515]:

# cond=movie['Year']>2015
# movie.loc[cond,['Netflix','Hulu', 'Prime Video', 'Disney+']]
#----------------------------------------------------------------
cond=Platforms_df1['Year']>2015
Platforms_df1.loc[cond,['Platform']].value_counts().head(1)
#-------------------------------------
Platforms_df1.loc[cond,['Platform']].mode()

,Platform
0,Prime Video


# What are the top 5 genres with the most movies?

In [516]:

movie['Genres'].value_counts().head()

#------------------or-----------------



Genres
Drama           1341
Documentary     1229
Comedy          1040
Comedy,Drama     446
Horror           436
Name: count, dtype: int64

In [517]:
# Genres_df['Genres'].value_counts().head()


# Which genre has the highest average IMDb rating?

In [518]:

gp=Genres_df.groupby('Genres')
gp['IMDb'].mean().idxmax()

'News'

# What is the most popular genre on Netflix?

In [519]:

cond=movie['Netflix']==1
movie.loc[cond,['Genres']].mode()

,Genres
0,Comedy


# How many movies in the Comedy genre have an IMDb rating above 7.5?

In [520]:

cond=(Genres_df['Genres']=='Comedy') & (Genres_df['IMDb']>7.5)
Genres_df.loc[cond,['Title']].value_counts().count()

np.int64(316)

# What is the average IMDb rating for Horror movies?

In [521]:

cond=Genres_df['Genres']=='Horror'
Genres_df.loc[cond,'IMDb'].mean()

np.float64(4.730713312130849)

# Who are the top 5 directors with the most movies in the dataset?

In [522]:

Directors_df['Directors'].value_counts()

Directors
unknown           726
Jay Chapman        36
Joseph Kane        31
Cheh Chang         29
Jim Wynorski       23
                 ... 
Frankie Chung       1
Paul Kenworthy      1
Ralph Wright        1
David Grossman      1
Mark Davis          1
Name: count, Length: 12454, dtype: int64

# Which director has the highest average IMDb rating?

In [523]:

# gp=Directors_df.groupby(['Directors'])
# gp['IMDb'].mean().idxmax()
#--------------------------------------------------
gp=movie.groupby(['Directors'])
gp['IMDb'].mean().idxmax()

'Danny Wu'

# Which director has made the most Action movies?

In [524]:
pd.concat([Platforms_df,Genres_df])
dir_ge=pd.merge(left=Genres_df,right=Directors_df,how='left' ,on='Title')
dir_ge
cond=dir_ge['Genres']=='Action' 
dir_ge.loc[cond,['Directors']].value_counts()
dir_ge.loc[cond,['Directors']].mode()


#--------------------------------------------------------------------------------


# movie.loc[(movie['Genres']=='Action'),'Directors'].value_counts().head()



,Directors
0,unknown


# What is the average IMDb rating of movies directed by Christopher Nolan?

In [525]:
cond=movie['Directors']=='Christopher Nolan'
movie.loc[cond,['IMDb']].mean()

IMDb    8.3
dtype: float64


# What are the top 5 most common languages in the dataset

In [526]:
Languages_df['Language'].value_counts().head()

Language
English    13233
Spanish      872
French       799
Hindi        731
unknown      614
Name: count, dtype: int64

# What is the average IMDb rating of French movies?

In [527]:
cond=Languages_df['Language']=="French"
Languages_df.loc[cond,['IMDb']].mean()
#-------------------or------------------------------
cond=movie['Language']=="French"
movie.loc[cond,['IMDb']].mean()

IMDb    6.223926
dtype: float64

# How many movies are from the United States?

In [528]:
cond=movie['Country']=='United States'
movie.loc[cond,['Country']].value_counts()

Country      
United States    8776
Name: count, dtype: int64

# Which country has produced the most Drama movies?

In [529]:
cond=movie['Genres']=='Drama'
movie.loc[cond,['Country']].mode()
#----------------or------------------------
cond=Country_df['Genres']=='Drama'
Country_df.loc[cond,['Country','Genres']].value_counts()
Country_df.loc[cond,['Country']].mode()

,Country
0,United States


 # What is the most common language for movies on Hulu?

In [530]:
#-------------or--------------------------------
cond=movie['Hulu']==1
movie.loc[cond,'Language'].mode()

0    English
Name: Language, dtype: object

# IMDb and Rotten Tomatoes Score Analysis

In [531]:
movie.loc[::,['IMDb','Rotten Tomatoes']].corr()

,IMDb,Rotten Tomatoes
IMDb,1.000000,0.261922
Rotten Tomatoes,0.261922,1.000000


# How many movies by Steven Spielberg are available on Prime Video?

In [532]:
#-------------or-----------------------------------
cond=(movie['Directors']=='Steven Spielberg')&(movie['Prime Video']==1)
movie.loc[cond,['Title']].count()

Title    1
dtype: int64

# What is the highest IMDb rating in the dataset, and which movie has it?

In [537]:

# max(movie['IMDb'])   #9.3
# cond=movie['IMDb']>=9.3
# movie.loc[cond,['Title','IMDb']].idxmax()
#--------------or----------
cond=movie['IMDb'].idxmax()
movie.loc[cond,['Title','IMDb']]

Title    My Next Guest with David Letterman and Shah Ru...
IMDb                                                   9.3
Name: 1292, dtype: object

# What is the lowest Rotten Tomatoes score, and which movie has it?

In [536]:
# min(movie['Rotten Tomatoes']) #0.2
# cond=movie['Rotten Tomatoes']<=0.2
# movie.loc[cond,['Title','Rotten Tomatoes']]
#--------------or----------------
cond=movie['Rotten Tomatoes'].idxmin()
movie.loc[cond,['Title','Rotten Tomatoes']]

Title              Getaway
Rotten Tomatoes        0.2
Name: 4208, dtype: object

# What is the correlation between IMDb ratings and Rotten Tomatoes scores?

In [ ]:
movie.loc[::,['IMDb','Rotten Tomatoes']].corr()

,IMDb,Rotten Tomatoes
IMDb,1.000000,0.261922
Rotten Tomatoes,0.261922,1.000000


# How many movies have both IMDb rating above 8.0 and Rotten Tomatoes score above 90%?

In [ ]:
cond=(movie['IMDb']>8.0) & (movie['Rotten Tomatoes']>9.0)
movie.loc[cond,['Title','IMDb','Rotten Tomatoes']]

,Title,IMDb,Rotten Tomatoes
3,Back to the Future,8.5,9.6
4,"The Good, the Bad and the Ugly",8.8,9.7
5,Spider-Man: Into the Spider-Verse,8.4,9.7
6,The Pianist,8.5,9.5
8,Raiders of the Lost Ark,8.4,9.5
...,...,...,...
16220,Finding Nemo,8.1,9.9
16221,Coco,8.4,9.7
16222,Toy Story 3,8.3,9.8
16229,The Princess Bride,8.1,9.7


# Which platform has the highest average Rotten Tomatoes score?

In [ ]:

gp=Platforms_df1.groupby(['Platform'])
gp['Rotten Tomatoes'].mean().idxmax()


'Hulu'

# What is the average runtime of all movies?

In [ ]:
movie['Runtime'].mean()

np.float64(93.4134472511144)

# Which movie has the longest runtime?

In [ ]:
cond=(movie['Runtime']==max(movie['Runtime']))
movie.loc[cond,['Title','Runtime']]

,Title,Runtime
13179,Colorado,1256.0


# Which genre tends to have the longest average runtime 

In [ ]:
Genres_df.groupby(['Genres'])['Runtime'].mean().idxmax()

'Musical'

# Which genre tends to have the longest  runtime on avg 

In [ ]:
# Genres_df.groupby(['Genres'])['Runtime'].max().mean()


In [ ]:

cond=max(movie['Runtime'])
Genres_df.loc[cond,['Genres']]
#---------------or------------
movie.loc[cond,['Genres']]

Genres    Drama,Romance
Name: 1256, dtype: object

# How many movies have a runtime of less than 60 minutes?

In [ ]:
cond=movie['Runtime']<60
movie.loc[cond,['Title']].count()

Title    1219
dtype: int64

# What is the average IMDb rating for movies with a runtime above 150 minutes?

In [ ]:
cond=movie['Runtime']>150
movie.loc[cond,['IMDb']].mean()

IMDb    6.622574
dtype: float64

# How many unique movies are in the dataset?
# What is the average IMDb rating of all movies?
# How many movies were released each year?
# What is the most common genre in the dataset?
# What is the most common language for movies?

# How many movies are available on each streaming platform?
# Which platform has the highest average IMDb rating?
# Are there any movies available on all four platforms (Netflix, Hulu, Prime Video, Disney+)?
# Which platform has the most movies with an IMDb rating above 8.0?
# Which platform has the most movies released after 2015?

# What are the top 5 genres with the most movies?
# Which genre has the highest average IMDb rating?
# What is the most popular genre on Netflix?
# How many movies in the Comedy genre have an IMDb rating above 7.5?
# What is the average IMDb rating for Horror movies?

# Who are the top 5 directors with the most movies in the dataset?
# Which director has the highest average IMDb rating?
# Which director has made the most Action movies?
# What is the average IMDb rating of movies directed by Christopher Nolan?
# How many movies by Steven Spielberg are available on Prime Video?

# What are the top 5 most common languages in the dataset?
# What is the average IMDb rating of French movies?
# How many movies are from the United States?
# Which country has produced the most Drama movies?
# What is the most common language for movies on Hulu?
# IMDb and Rotten Tomatoes Score Analysis
# What is the highest IMDb rating in the dataset, and which movie has it?
# What is the lowest Rotten Tomatoes score, and which movie has it?
# What is the correlation between IMDb ratings and Rotten Tomatoes scores?
# How many movies have both IMDb rating above 8.0 and Rotten Tomatoes score above 90%?
# Which platform has the highest average Rotten Tomatoes score?

# What is the average runtime of all movies?
# Which movie has the longest runtime?
# Which genre tends to have the longest runtime on average?
# How many movies have a runtime of less than 60 minutes?
# What is the average IMDb rating for movies with a runtime above 150 minutes?